In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def random_image(val_dir,val_class):
    folder = val_dir + val_class
    random_image = random.sample(os.listdir(folder),1)
    img = mpimg.imread(folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(val_class)
    return img

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Создание наборов данных из изображений

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dir = "/content/drive/MyDrive/Data/train"
test_dir =  "/content/drive/MyDrive/Data/test"
val_dir = "/content/drive/MyDrive/Data/val"

train_data = image_dataset_from_directory(train_dir,label_mode = "categorical", image_size = (224,224),batch_size = 16, shuffle = True,seed = 42)
test_data = image_dataset_from_directory(test_dir,label_mode = "categorical", image_size = (224,224),batch_size = 16, shuffle = False,seed = 42)
val_data = image_dataset_from_directory(val_dir,label_mode = "categorical", image_size = (224,224),batch_size = 16, shuffle = False,seed = 42)

*Обратный вызов*

EarlyStopping - Этот обратный вызов полезен для предотвращения переобучения: если значение функции потерь на валидационном наборе данных не улучшается в течение определенного числа эпох, обучение прекращается.

ReduceLROnPlateau - Этот обратный вызов полезен для динамического управления шагом обучения: если функция потерь на валидационном наборе данных не улучшается в течение определенного числа эпох, шаг обучения уменьшается, что может помочь в сходимости модели.

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",patience = 5, min_delta = 0.0001)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss",factor = 0.2, patience = 4, min_lr = 1e-7)

Аугментация данных

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
data_aug = tf.keras.Sequential([
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomContrast(factor=0.2),
    preprocessing.RandomFlip("vertical"),
],name = "data_augmentation_layer")

 EfficientNetB0 - это предварительно обученная модель глубокого обучения, которая обеспечивает эффективную архитектуру для извлечения признаков из изображений

In [ ]:
model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='/content/drive/MyDrive/Data/efficientnetb0_notop.h5',
)
model.trainable = False

Создаем конечную модель, объединяя аугментированный вход, модель EfficientNetB0

In [ ]:
inputs = layers.Input(shape = (224,224,3),name = "input_layer")
x = data_aug(inputs)
x = model(x)
x = layers.GlobalAvgPool2D(name = "pooling_layer")(x)
x = layers.Dense(256, activation=tf.keras.layers.LeakyReLU(alpha=0.2), kernel_initializer=tf.keras.initializers.he_normal())(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6)(x)
outputs = layers.Activation("softmax",dtype = tf.float32)(x)
model = tf.keras.Model(inputs,outputs)
model.summary()

Компиляция модели

In [ ]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer = tf.keras.optimizers.Adam(),metrics = ["accuracy"])

Обучение модели с использованием данных обучения train_data и валидации val_data


In [ ]:
history = model.fit(train_data,epochs = 13 , validation_data = val_data, callbacks = [early_stop,reduce_lr])

In [ ]:
print("Testing Accuracy",model.evaluate(test_data))

Сохраняем модель

In [ ]:
model.save('DermAI_v10.h5')